In [1]:
from lib.EmbeddingProvider import OpenAiEmbeddingProvider

from lib.DataRepository import DataRepository
from lib.questions import QuestionExtractor

repo = DataRepository(embedding=OpenAiEmbeddingProvider(), db_path="./data/db/open_ai_small_50_10", chunk_size=50, chunk_overlap=10)
# repo.save_by_file()

# Parce question and find metrics in the question, find similar metrics

In [2]:
from lib.questions import QuestionExtractor

question = "Which leadership **positions** changed at Canadian Tire Corporation in the reporting period? If data is not available, return 'N/A'."
kind = "names"

extractor = QuestionExtractor()
extract = extractor.extract(question)
print(extract)
close_metrics = extractor.get_synonyms(extract["metric"])
print(close_metrics)

{'original_question': "Which leadership **positions** changed at Canadian Tire Corporation in the reporting period? If data is not available, return 'N/A'.", 'metric': 'leadership positions changed', 'companies': ['Canadian Tire Corporation'], 'currency': None, 'comparison': None, 'category': 'leadership_changed'}
['changed leadership roles', 'modified executive positions', 'altered leadership roles', 'leadership role modifications']


In [3]:
import json

companiy = extract["companies"][0]
with open("data/r2.0-test/subset.json", 'r') as file:
    subset = json.load(file)
filered = list(filter(lambda x: x["company_name"] == companiy, subset))[0]
filered

{'sha1': '7c55d7900a241e732c145687598d43c915a678f9',
 'cur': 'USD',
 'company_name': 'Canadian Tire Corporation',
 'major_industry': 'Retail',
 'mentions_recent_mergers_and_acquisitions': True,
 'has_leadership_changes': True,
 'has_layoffs': False,
 'has_executive_compensation': True,
 'has_rnd_investment_numbers': True,
 'has_new_product_launches': True,
 'has_capital_expenditures': True,
 'has_financial_performance_indicators': True,
 'has_dividend_policy_changes': True,
 'has_share_buyback_plans': True,
 'has_capital_structure_changes': False,
 'mentions_new_risk_factors': True,
 'has_guidance_updates': True,
 'has_regulatory_or_litigation_issues': False,
 'has_strategic_restructuring': False,
 'has_supply_chain_disruptions': True,
 'has_esg_initiatives': True}

# Find similar metrics in the data base

In [9]:
main_metric = extract["metric"]
print(main_metric)
file_filter = { "source" : f"./data/r2.0-test/pdfs/{filered["sha1"]}.pdf" }
main_results = repo.query(main_metric, k=10, f=file_filter) # start with main metric from the question

main_metric = extract["metric"]
smaller_results = [] # start with main metric from the question
for m in close_metrics:
    smaller_results += repo.query(m, k=5) # find similar metrics
search_results = main_results + smaller_results
print(len(search_results))

leadership positions changed
30


In [ ]:
filtered_search